## Notebook to compare total demand by TAZ for the transit mode in two different scenarios

In [ ]:
# Notebook to compare total demand by TAZ for the transit mode in two different scenarios

import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

### User input required: Specify paths to input and output directories in config.py file

In [ ]:
%run "S:/jupyter_notebooks/config.py"

### User input required: Name of CSV file with report output for "base" scenario

In [ ]:
# Supply name of CSV file with report output for "base" scenario:
#
base_csv_fn = 'taz_transit_report_base.csv'

### User input required: Name of CSV file with report output for "comparison" scenario

In [ ]:
# Supply name of CSV file with report for "comparison" scenario:
#
comp_csv_fn = 'taz_transit_report_comp.csv'

###  User input required: Name of CSV output file generated by this notebook

In [ ]:
# Supply name of CSV output file for report generated by this notebook:
#
csv_output_fn = 'taz_transit_report_DIFF.csv'

In [ ]:
fq_base_csv_fn = sandbox_dir + base_csv_fn
fq_comp_csv_fn = sandbox_dir + comp_csv_fn

In [ ]:
# Columns containing "reference" data
#
ref_column_names = [ 'id', 'town', 'state' ]
#
# Columns containing "metric" data
#
metric_column_names = [ 'transit_total', 'boat', 'cr', 'lb', 'rt', 'wat' ]
#
all_column_names = ref_column_names + metric_column_names

### Read data for "base" scenario

In [ ]:
# Read "base" data into a dataframe
#
base_df = pd.read_csv(fq_base_csv_fn, delimiter=',', usecols=all_column_names)

In [ ]:
base_df

### Read data for "comparison" scenario

In [ ]:
# Read "comparison" data into a dataframe
comp_df = pd.read_csv(fq_comp_csv_fn, delimiter=',',  usecols=all_column_names)

In [ ]:
comp_df

In [ ]:
# Create a dataframe in which to compute the delta between "comp" and "base"
delta_df = pd.merge(left=comp_df, right=base_df, on="id", suffixes=('_comp', '_base'))

In [ ]:
delta_df

### Compute delta: comparison - base

In [ ]:
# Compute the detlta between the two dataframes: comp - base
# Calculate the delta (comparision - base) for each metric
#
for column_name in metric_column_names:
    base_column_name = column_name + '_base'
    comp_column_name = column_name + '_comp'
    delta_column_name = column_name + '_delta'
    delta_df[delta_column_name] = delta_df[comp_column_name] - delta_df[base_column_name]
# end_for

In [ ]:
delta_df

In [ ]:
taz_shapefile_base_dir = reference_data_dir + 'canonical_TAZ_shapefile/'

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(delta_df.set_index('id'), on='id')

### Export CSV report output

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = sandbox_dir + csv_output_fn
delta_column_names = [ cn + '_delta' for cn in metric_column_names]
cols = ref_column_names + delta_column_names
joined_df.to_csv(fq_output_fn, sep=',', columns=cols)

### Make static and interactive maps of results

In [ ]:
# Make a static map of total auto trips by origin TAZ
joined_df.plot("transit_total_delta", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Change in Total Transit Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='transit_total_delta', 
                 geo=True, 
                 hover_cols=cols, 
                 clabel='Change in Total Transit Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Change in Total Transit Trips by Origin TAZ')